## QA Products full data

This notebook fetches the full data of each of the product we have in our products for QA.
This notebook also corrects the invalid product_asin that is in some of the qa our synthetic products has

In [2]:
%pip install --quiet python-dotenv

from dotenv import load_dotenv
load_dotenv()




[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


True

In [3]:
from copy import deepcopy
from pathlib import Path
import json

product_path = "../initial-training-sets/datasets/"
qa_json_filenames = ["men_fashion_qa.json", "skin_care_qa.json", "women_fashion_qa.json"]

dataset_products = {}
for qa_json_filename in qa_json_filenames:
    qa_product_list = json.loads(Path(product_path + qa_json_filename).read_text())
    
    for qa_product in qa_product_list:
        if qa_product['category'] not in dataset_products:
            dataset_products[qa_product['category']] = {
                "products": [],
                "qa": []
            }

        _qa_product = deepcopy(qa_product)
        # del _qa_product['product_asin']
        dataset_products[qa_product['category']]['qa'].append(_qa_product)
        raw_products_path = Path(f"../initial-training-sets/{qa_product['category']}/raw_products.json")
        # cache function? lru_cache?
        raw_products = json.loads(Path(raw_products_path).read_text())
        
        matched_products = [ raw_product for raw_product in raw_products if raw_product['product_asin'] == qa_product['product_asin'] or qa_product['product_asin'] in raw_product['description'] ]
        dataset_products[qa_product['category']]['products'].extend(matched_products)
            

In [4]:
# Correct the product_asin in the qa list
corrected_dataset_products = {}
for index, category in enumerate(dataset_products):

    products = dataset_products[category]['products']
    qa = dataset_products[category]['qa']
    
    new_qa = []
    for qa_product in qa:
        correct_product = [product for product in products if qa_product['product_asin'] != product['product_asin'] and qa_product['product_asin'] in product['description']]
        if correct_product:
            _qa_product = deepcopy(qa_product)
            _qa_product['product_asin'] = correct_product[0]['product_asin']
            new_qa.append(_qa_product)
        else:
            new_qa.append(qa_product)
            
    corrected_dataset_products[category]  = {
            "products": products,
            "qa": new_qa
        }
    
    

In [5]:
# WOMEN_FASHION_NO_REVIEWS, MEN_FASHION_WITH_REVIEWS, SKIN_CARE_WITH_REVIEWS

In [6]:
len(corrected_dataset_products['WOMEN_FASHION_NO_REVIEWS']['products']), len(corrected_dataset_products['WOMEN_FASHION_NO_REVIEWS']['qa'])

(25, 40)

In [7]:
len(corrected_dataset_products['MEN_FASHION_WITH_REVIEWS']['products']), len(corrected_dataset_products['MEN_FASHION_WITH_REVIEWS']['qa'])

(21, 24)

In [8]:
len(corrected_dataset_products['SKIN_CARE_WITH_REVIEWS']['products']), len(corrected_dataset_products['SKIN_CARE_WITH_REVIEWS']['qa'])

(6, 7)

## Save the products to a file

In [9]:
# un-comment this if you do not have the corrected products 
# Path("../initial-training-sets/datasets/dataset_products.json").write_text("")
# Path("../initial-training-sets/datasets/dataset_products.json").write_text(json.dumps(corrected_dataset_products))

## Move the corresponding product images as well

In [32]:
import os
import shutil
dataset_images_dir_path = Path("../initial-training-sets/datasets/images")
if not dataset_images_dir_path.exists():
    os.makedirs(dataset_images_dir_path, exist_ok=True)

for title in corrected_dataset_products:
    products = corrected_dataset_products[title]['products']
    raw_products_path = Path(f"../initial-training-sets/{title}/images")
    for product in products:
        for image_path in [path for path in os.listdir(raw_products_path.absolute()) if product['product_asin'] in path]:
            shutil.copy(str(raw_products_path.absolute()) +  "/" + image_path, str(dataset_images_dir_path))